In [94]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFilter
import json
import csv

Annotating a Sample Image

In [95]:
epidural_csv_file = 'segmentations/Results_Epidural Hemorrhage Detection_2020-11-16_21.31.26.148.csv'
epidural_image_repo = 'epidural/max_contrast_window'
epidural_image_file = 'ID_004c4b319.jpg'

In [96]:
def get_coords(image, image_csv_file, image_file_name):
    labels = get_labels_csv(image_csv_file, image_file_name)
    coord_list = []
    for label in labels:
        coord = (label['x'] * image.size[0], label['y'] * image.size[1])
        coord_list.append(coord)
    return coord_list
def get_labels_csv(image_csv_file, image_file_name):
    with open(image_csv_file) as csv_file:
        data_reader = csv.reader(csv_file, dialect='excel')
        for row in data_reader:
            if row[1] == image_file_name:
                return json.loads(row[7])
    return None

In [97]:
im = Image.open(f'{epidural_image_repo}/{epidural_image_file}')
features = []
for x in range(im.size[0]):
    for y in range(20):
        features.append(im.getpixel((x, y))[0])
        features.append(im.getpixel((x, y))[1])
        features.append(im.getpixel((x, y))[2])
draw = ImageDraw.Draw(im)
draw.line(get_coords(im, epidural_csv_file, epidural_image_file), fill='red')
im.show()

In [98]:
# the line of code below loads the 'all labels' field in the csv file in json
# coords_lists = json.loads(row[4].replace("'[", "[").replace("]'", "],").replace(',]', ']'))

Reading an Image and Its Classification

In [99]:
num_features = 786432

In [100]:
all_labels = pd.read_csv('segmentations/hemorrhage-labels.csv')

In [101]:
all_labels

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_000012eaf,0,0,0,0,0,0
1,ID_000039fa0,0,0,0,0,0,0
2,ID_00005679d,0,0,0,0,0,0
3,ID_00008ce3c,0,0,0,0,0,0
4,ID_0000950d7,0,0,0,0,0,0
...,...,...,...,...,...,...,...
752798,ID_ffff82e46,0,0,0,0,0,0
752799,ID_ffff922b9,1,0,0,1,0,0
752800,ID_ffffb670a,1,0,0,0,1,0
752801,ID_ffffcbff8,0,0,0,0,0,0


In [102]:
labels = all_labels[all_labels['any'] == 1]

In [103]:
labels

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
14,ID_0002081b6,1,0,1,0,0,0
24,ID_0002a38ad,1,0,0,0,1,1
33,ID_000346ce2,1,0,0,0,0,1
36,ID_00042829c,1,0,0,1,0,0
43,ID_0004a5701,1,0,0,0,0,1
...,...,...,...,...,...,...,...
752755,ID_fffc60817,1,0,1,1,0,0
752769,ID_fffd00949,1,0,0,0,1,0
752783,ID_fffe2edb8,1,0,1,1,0,0
752799,ID_ffff922b9,1,0,0,1,0,0


In [128]:
epidural = labels[(labels['epidural'] == 1) & 
                  (labels['intraparenchymal'] == 0) &
                  (labels['intraventricular'] == 0) & 
                  (labels['subarachnoid'] == 0) &
                  (labels['subdural'] == 0)] 

In [129]:
epidural

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
164,ID_000edbf38,1,1,0,0,0,0
271,ID_00178eb80,1,1,0,0,0,0
320,ID_001bb2c00,1,1,0,0,0,0
443,ID_0026de01c,1,1,0,0,0,0
829,ID_004966e2d,1,1,0,0,0,0
...,...,...,...,...,...,...,...
751555,ID_ff903bbeb,1,1,0,0,0,0
751800,ID_ffa6e19a5,1,1,0,0,0,0
752300,ID_ffd27f702,1,1,0,0,0,0
752334,ID_ffd67050a,1,1,0,0,0,0


In [130]:
epidural['Image']

164       ID_000edbf38
271       ID_00178eb80
320       ID_001bb2c00
443       ID_0026de01c
829       ID_004966e2d
              ...     
751555    ID_ff903bbeb
751800    ID_ffa6e19a5
752300    ID_ffd27f702
752334    ID_ffd67050a
752455    ID_ffe17f20d
Name: Image, Length: 1694, dtype: object

In [131]:
epidural.shape

(1694, 7)

In [132]:
epidural_data = np.zeros((epidural.shape[0], num_features))

In [133]:
epidural_data.shape

(1694, 786432)

In [134]:
for index, value in enumerate(epidural['Image'].items()):
    print(f'epidural/max_contrast_window/{value[1]}.jpg')
    epidural_data[index, :] = plt.imread(f'epidural/max_contrast_window/{value[1]}.jpg').reshape(1,-1)

epidural/max_contrast_window/ID_000edbf38.jpg
epidural/max_contrast_window/ID_00178eb80.jpg
epidural/max_contrast_window/ID_001bb2c00.jpg
epidural/max_contrast_window/ID_0026de01c.jpg
epidural/max_contrast_window/ID_004966e2d.jpg
epidural/max_contrast_window/ID_004c4b319.jpg
epidural/max_contrast_window/ID_00a05276d.jpg
epidural/max_contrast_window/ID_00c13f51d.jpg
epidural/max_contrast_window/ID_00f1e66e1.jpg
epidural/max_contrast_window/ID_0125f211e.jpg
epidural/max_contrast_window/ID_013f798f9.jpg
epidural/max_contrast_window/ID_019d29542.jpg
epidural/max_contrast_window/ID_0237b835f.jpg
epidural/max_contrast_window/ID_02c9be37e.jpg
epidural/max_contrast_window/ID_02e2d273d.jpg
epidural/max_contrast_window/ID_02f37ce84.jpg
epidural/max_contrast_window/ID_0304d5570.jpg
epidural/max_contrast_window/ID_0317e342a.jpg
epidural/max_contrast_window/ID_036ac8037.jpg
epidural/max_contrast_window/ID_036daa1a7.jpg
epidural/max_contrast_window/ID_03755d5a6.jpg
epidural/max_contrast_window/ID_03

epidural/max_contrast_window/ID_1d01678ef.jpg
epidural/max_contrast_window/ID_1d053d6dd.jpg
epidural/max_contrast_window/ID_1d17eee34.jpg
epidural/max_contrast_window/ID_1d3a3e2a8.jpg
epidural/max_contrast_window/ID_1d5a459c9.jpg
epidural/max_contrast_window/ID_1d8bdd15f.jpg
epidural/max_contrast_window/ID_1d9a44cbc.jpg
epidural/max_contrast_window/ID_1daab912b.jpg
epidural/max_contrast_window/ID_1db17b608.jpg
epidural/max_contrast_window/ID_1de9cf2cf.jpg
epidural/max_contrast_window/ID_1df27d911.jpg
epidural/max_contrast_window/ID_1e06cb426.jpg
epidural/max_contrast_window/ID_1e35d103d.jpg
epidural/max_contrast_window/ID_1ef06a053.jpg
epidural/max_contrast_window/ID_1f054a3fd.jpg
epidural/max_contrast_window/ID_1f0ae213d.jpg
epidural/max_contrast_window/ID_1f2aad150.jpg
epidural/max_contrast_window/ID_1f2f4ec16.jpg
epidural/max_contrast_window/ID_1f69053d4.jpg
epidural/max_contrast_window/ID_1f9c6bb18.jpg
epidural/max_contrast_window/ID_1ff1acc28.jpg
epidural/max_contrast_window/ID_20

epidural/max_contrast_window/ID_38723757a.jpg
epidural/max_contrast_window/ID_38ad3a242.jpg
epidural/max_contrast_window/ID_397fd654d.jpg
epidural/max_contrast_window/ID_399a1ed27.jpg
epidural/max_contrast_window/ID_39d2b7c29.jpg
epidural/max_contrast_window/ID_3a13e18a5.jpg
epidural/max_contrast_window/ID_3a3be84f6.jpg
epidural/max_contrast_window/ID_3a42ff4e3.jpg
epidural/max_contrast_window/ID_3a4766818.jpg
epidural/max_contrast_window/ID_3a4e45124.jpg
epidural/max_contrast_window/ID_3a738fab5.jpg
epidural/max_contrast_window/ID_3ac31747b.jpg
epidural/max_contrast_window/ID_3ae9a6aa2.jpg
epidural/max_contrast_window/ID_3b24b9be4.jpg
epidural/max_contrast_window/ID_3b2a589c0.jpg
epidural/max_contrast_window/ID_3b48698b8.jpg
epidural/max_contrast_window/ID_3b5344b9c.jpg
epidural/max_contrast_window/ID_3c0dca0a9.jpg
epidural/max_contrast_window/ID_3c0ebd8a1.jpg
epidural/max_contrast_window/ID_3c17fc4ba.jpg
epidural/max_contrast_window/ID_3c22d2941.jpg
epidural/max_contrast_window/ID_3c

epidural/max_contrast_window/ID_53a3527f3.jpg
epidural/max_contrast_window/ID_53b28e053.jpg
epidural/max_contrast_window/ID_53b8c832d.jpg
epidural/max_contrast_window/ID_53fc14977.jpg
epidural/max_contrast_window/ID_5411705c5.jpg
epidural/max_contrast_window/ID_5414b1b78.jpg
epidural/max_contrast_window/ID_5467c2693.jpg
epidural/max_contrast_window/ID_54995c469.jpg
epidural/max_contrast_window/ID_549b3e4c2.jpg
epidural/max_contrast_window/ID_54ab0e914.jpg
epidural/max_contrast_window/ID_54abaadcd.jpg
epidural/max_contrast_window/ID_54b494cca.jpg
epidural/max_contrast_window/ID_54c71383b.jpg
epidural/max_contrast_window/ID_54d5c3d88.jpg
epidural/max_contrast_window/ID_54f55b16f.jpg
epidural/max_contrast_window/ID_5503418f9.jpg
epidural/max_contrast_window/ID_55499add6.jpg
epidural/max_contrast_window/ID_5565bf680.jpg
epidural/max_contrast_window/ID_5568e4fb2.jpg
epidural/max_contrast_window/ID_55a0dae9b.jpg
epidural/max_contrast_window/ID_55a1e893d.jpg
epidural/max_contrast_window/ID_55

epidural/max_contrast_window/ID_71a8f7888.jpg
epidural/max_contrast_window/ID_71b6836d4.jpg
epidural/max_contrast_window/ID_71bcb99cc.jpg
epidural/max_contrast_window/ID_72764fc1c.jpg
epidural/max_contrast_window/ID_7278f42f0.jpg
epidural/max_contrast_window/ID_7292bbba9.jpg
epidural/max_contrast_window/ID_72a49b64c.jpg
epidural/max_contrast_window/ID_72c4efbfe.jpg
epidural/max_contrast_window/ID_72fa166e8.jpg
epidural/max_contrast_window/ID_73088ca71.jpg
epidural/max_contrast_window/ID_7322034ba.jpg
epidural/max_contrast_window/ID_7339c1a56.jpg
epidural/max_contrast_window/ID_7347b4d0b.jpg
epidural/max_contrast_window/ID_73af55fe6.jpg
epidural/max_contrast_window/ID_73e28d4e2.jpg
epidural/max_contrast_window/ID_7438a6a4d.jpg
epidural/max_contrast_window/ID_744220ea2.jpg
epidural/max_contrast_window/ID_74d1456af.jpg
epidural/max_contrast_window/ID_75079dbe5.jpg
epidural/max_contrast_window/ID_759cc0dc1.jpg
epidural/max_contrast_window/ID_759fe2429.jpg
epidural/max_contrast_window/ID_76

epidural/max_contrast_window/ID_89b065f11.jpg
epidural/max_contrast_window/ID_89d9ec60a.jpg
epidural/max_contrast_window/ID_8a1af3f75.jpg
epidural/max_contrast_window/ID_8a2c71f24.jpg
epidural/max_contrast_window/ID_8a2e8d234.jpg
epidural/max_contrast_window/ID_8a68bfa74.jpg
epidural/max_contrast_window/ID_8a71e0149.jpg
epidural/max_contrast_window/ID_8abe5dfd5.jpg
epidural/max_contrast_window/ID_8b00af438.jpg
epidural/max_contrast_window/ID_8b0c9ee8c.jpg
epidural/max_contrast_window/ID_8b962ef76.jpg
epidural/max_contrast_window/ID_8bc7fb632.jpg
epidural/max_contrast_window/ID_8bd79f5c7.jpg
epidural/max_contrast_window/ID_8bef1f190.jpg
epidural/max_contrast_window/ID_8c04493a5.jpg
epidural/max_contrast_window/ID_8c1152997.jpg
epidural/max_contrast_window/ID_8c330029e.jpg
epidural/max_contrast_window/ID_8c3697c0c.jpg
epidural/max_contrast_window/ID_8c4b04f4d.jpg
epidural/max_contrast_window/ID_8c8fd4866.jpg
epidural/max_contrast_window/ID_8cadb065b.jpg
epidural/max_contrast_window/ID_8c

epidural/max_contrast_window/ID_a4e2930de.jpg
epidural/max_contrast_window/ID_a55d0f15b.jpg
epidural/max_contrast_window/ID_a59b1ba40.jpg
epidural/max_contrast_window/ID_a6252ad05.jpg
epidural/max_contrast_window/ID_a6290b2a6.jpg
epidural/max_contrast_window/ID_a62a4aeee.jpg
epidural/max_contrast_window/ID_a62ad69dd.jpg
epidural/max_contrast_window/ID_a6514fed2.jpg
epidural/max_contrast_window/ID_a6bf37726.jpg
epidural/max_contrast_window/ID_a75babb6a.jpg
epidural/max_contrast_window/ID_a782068a9.jpg
epidural/max_contrast_window/ID_a78646919.jpg
epidural/max_contrast_window/ID_a7b71eae6.jpg
epidural/max_contrast_window/ID_a7e1f547a.jpg
epidural/max_contrast_window/ID_a8468ede9.jpg
epidural/max_contrast_window/ID_a875a0c1c.jpg
epidural/max_contrast_window/ID_a89362cf0.jpg
epidural/max_contrast_window/ID_a8b23e5ff.jpg
epidural/max_contrast_window/ID_a8b33969f.jpg
epidural/max_contrast_window/ID_a8b817d20.jpg
epidural/max_contrast_window/ID_a8f16d26b.jpg
epidural/max_contrast_window/ID_a9

epidural/max_contrast_window/ID_c0ad857b6.jpg
epidural/max_contrast_window/ID_c0d0e2d2e.jpg
epidural/max_contrast_window/ID_c0f7c1271.jpg
epidural/max_contrast_window/ID_c120311d3.jpg
epidural/max_contrast_window/ID_c12e40135.jpg
epidural/max_contrast_window/ID_c1687e324.jpg
epidural/max_contrast_window/ID_c16de5ece.jpg
epidural/max_contrast_window/ID_c215040c9.jpg
epidural/max_contrast_window/ID_c2195288f.jpg
epidural/max_contrast_window/ID_c21baec65.jpg
epidural/max_contrast_window/ID_c22a3ce84.jpg
epidural/max_contrast_window/ID_c23e3ae90.jpg
epidural/max_contrast_window/ID_c27121afb.jpg
epidural/max_contrast_window/ID_c299ceefd.jpg
epidural/max_contrast_window/ID_c29c8850e.jpg
epidural/max_contrast_window/ID_c33d7ca95.jpg
epidural/max_contrast_window/ID_c391d168e.jpg
epidural/max_contrast_window/ID_c40bea3cf.jpg
epidural/max_contrast_window/ID_c40d97829.jpg
epidural/max_contrast_window/ID_c45664bc2.jpg
epidural/max_contrast_window/ID_c47cca293.jpg
epidural/max_contrast_window/ID_c4

epidural/max_contrast_window/ID_e0942808d.jpg
epidural/max_contrast_window/ID_e0977cddc.jpg
epidural/max_contrast_window/ID_e0cc5ee18.jpg
epidural/max_contrast_window/ID_e19bf5ed4.jpg
epidural/max_contrast_window/ID_e19bfbdbd.jpg
epidural/max_contrast_window/ID_e1a79991d.jpg
epidural/max_contrast_window/ID_e26a957e7.jpg
epidural/max_contrast_window/ID_e2fa71a93.jpg
epidural/max_contrast_window/ID_e30686f0c.jpg
epidural/max_contrast_window/ID_e394200ce.jpg
epidural/max_contrast_window/ID_e3ac5befa.jpg
epidural/max_contrast_window/ID_e3b219688.jpg
epidural/max_contrast_window/ID_e3bcc91d9.jpg
epidural/max_contrast_window/ID_e3f0a0d13.jpg
epidural/max_contrast_window/ID_e409f98fa.jpg
epidural/max_contrast_window/ID_e43ecd347.jpg
epidural/max_contrast_window/ID_e460ece98.jpg
epidural/max_contrast_window/ID_e4b7a3d3b.jpg
epidural/max_contrast_window/ID_e4f3a6e95.jpg
epidural/max_contrast_window/ID_e5087ec3f.jpg
epidural/max_contrast_window/ID_e5137693a.jpg
epidural/max_contrast_window/ID_e5

epidural/max_contrast_window/ID_ff0afaa64.jpg
epidural/max_contrast_window/ID_ff292b933.jpg
epidural/max_contrast_window/ID_ff359af80.jpg
epidural/max_contrast_window/ID_ff473454b.jpg
epidural/max_contrast_window/ID_ff49ed229.jpg
epidural/max_contrast_window/ID_ff4e07384.jpg
epidural/max_contrast_window/ID_ff55bb7f5.jpg
epidural/max_contrast_window/ID_ff5a1e41e.jpg
epidural/max_contrast_window/ID_ff63a0acd.jpg
epidural/max_contrast_window/ID_ff903bbeb.jpg
epidural/max_contrast_window/ID_ffa6e19a5.jpg
epidural/max_contrast_window/ID_ffd27f702.jpg
epidural/max_contrast_window/ID_ffd67050a.jpg
epidural/max_contrast_window/ID_ffe17f20d.jpg


In [136]:
epidural_data.shape

(1694, 786432)